In [2]:
import pandas as pd
import numpy as np  
import re, json, logging , tabula 
from lib.config import connstr
from lib.context import context
from lib.utils import *  
from lib.issue_provider import issue_provider 
from lib.script_generator import script_generator 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [11]:
def formatqt(s):
    s=s.replace('•','') 
    s=s.replace('','<br>')
    s=re.sub('[\n|\r]','',s) 
    return s  
l=formatqt(""" """) 
insert='\nINSERT INTO fsma_Questions (PK_Question, FormName, FK_QuestionGroup,sortpos,FK_PickListType, FK_QuestionType,identifier_text,ExternalLinkType, QuestionText, help_text) VALUES 	\n'

In [9]:
file=f'{ctx.get_down_dir()}ig.xlsx' 
converts={'Question':lambda x: re.sub('\n','',x) }
dff=pd.read_excel(file, converters=converts, sheet_name=int(0), header=0, usecols='A,P,Q,X,U,AC,AD,AI,AK,AQ,AP,AR' ) 

In [12]:
df=dff 
df = df[df["Question"].str.contains("Question") == False]
df["{QuestionText}"]=df["Question"]
 
df=df.fillna('').astype(str) 
 
df['ht']=df['Unnamed: 15']+df['Maturity Level'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['ht'].apply(lambda s: f'<li> <strong>Ad Hoc:</strong>{s}</li>') 
df.drop(columns=['Unnamed: 15','Maturity Level'], inplace=True) 

df['ht']=df['Unnamed: 20']+df['Unnamed: 23'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Defined:</strong>{s}</li>')
df.drop(columns=['Unnamed: 20','Unnamed: 23'], inplace=True)

df['ht']=df['Unnamed: 28']+df['Unnamed: 29'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Consistently Implemented:</strong>{s}</li>')
df.drop(columns=['Unnamed: 28','Unnamed: 29'], inplace=True)

df['ht']=df['Unnamed: 34']+df['Unnamed: 36'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Managed and Measurable:</strong>{s}</li>')
df.drop(columns=['Unnamed: 34','Unnamed: 36'], inplace=True)

df['ht']=df['Unnamed: 41']+df['Unnamed: 42']+df['Unnamed: 43'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Optimized:</strong>{s}</li>')
df.drop(columns=['Unnamed: 41','Unnamed: 42','Unnamed: 43'], inplace=True)
 
df.drop(columns=['ht', 'Question'], inplace=True)

df['{helptext}']=df['{helptext}'].apply(lambda s: f'<br/><br/><h6><b>Maturity Level:</b></h6><ul>{s}</ul>' if len(s) > 210 else '' )

df['{helptext}']=df['{helptext}'].apply(formatqt)
df['{QuestionText}']=df['{QuestionText}'].apply(formatqt)
df['{idt}']=df['{QuestionText}'].apply(lambda s: re.findall('(\d{1,2}.\d?\s{1,4})',s)[0].rstrip(" ").rstrip(".") )
df['{FK_PickListType}']='85' 
df['{FK_QuestionType}']=df['{QuestionText}'].apply(lambda s: '9' if "additional information" in s  else '17')

df['{QuestionText}']=df['{QuestionText}'].apply(lambda s: re.sub('\'','`',s) )
df['{helptext}']=df['{helptext}'].apply(lambda s: re.sub('\'','`',s) )
s=2
sort=0
pk=53009
for i,r, in df.iterrows():
    if re.search('Table \d{1,2}',r['{QuestionText}']):
        s=s+1
        sort=0
        pk=pk+10 
    else:
        if sort > 0:
            df.loc[i, '{pre}']=str(',')
        else:
            df.loc[i, '{pre}']=str(insert)
        sort=sort+1
        pk=pk+1
        df.loc[i, '{FK_QuestionGroup}']=f'@PK_QGroup+{s}'
        df.loc[i, '{sortpos}']=str(sort)
        df.loc[i, '{PK_Question}']=str(pk) 
df = df[df["{QuestionText}"].str.contains("Table ") == False]
df['{QuestionText}']=df['{QuestionText}'].apply(lambda s: re.sub('\d{1,2}.\d?\s{1,4}','',s).strip(" ") ) 
df[0:3] 


,{QuestionText},{helptext},{idt},{FK_PickListType},{FK_QuestionType},{pre},{FK_QuestionGroup},{sortpos},{PK_Question}
1,To what extent does the organization maintain ...,<br/><br/><h6><b>Maturity Level:</b></h6><ul><...,1,85,17,"\nINSERT INTO fsma_Questions (PK_Question, For...",@PK_QGroup+2,1,53010
2,To what extent does the organization use stand...,<br/><br/><h6><b>Maturity Level:</b></h6><ul><...,2,85,17,",",@PK_QGroup+2,2,53011
5,To what extent does the organization use stand...,<br/><br/><h6><b>Maturity Level:</b></h6><ul><...,3,85,17,",",@PK_QGroup+2,3,53012


In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}fsma_QuestionsInsert.sql') 
#print(code ) 

In [ ]:
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{code}' ) 

In [257]:
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'r', encoding='UTF-8') as f: 
    sql=f.read() 
m=re.search('--PARSEBEGIN(.*)--PARSEEND',sql, flags=re.DOTALL)
sql=sql.replace(m.group(1),code)
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 

#with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'w', encoding='UTF-8') as f: 
#    f.write() 
